<a href="https://colab.research.google.com/github/kimhwijin/KFood_Models/blob/master/KFood_Dataset_Unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import os
import zipfile
import numpy as np
import csv
import matplotlib.pyplot as plt

In [34]:
DATASET_PATH = Path(os.getcwd())

In [8]:
dest_path = DATASET_PATH / "kfood"
if not dest_path.exists():
    with zipfile.ZipFile(DATASET_PATH / "kfood.zip") as kfood:
        zipInfo = kfood.infolist()
        for member in zipInfo:
            try:
                print(member.filename.encode('cp437').decode('euc-kr', 'ignore'))
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                kfood.extract(member, dest_path)
            except:
                #print(source_file)
                raise Exception('what?!')


구이.zip
국.zip
기타.zip
김치.zip
나물.zip
떡.zip
만두.zip
면.zip
무침.zip
밥.zip
볶음.zip
쌈.zip
음청류.zip
장.zip
장아찌.zip
적.zip
전.zip
전골.zip
조림.zip
죽.zip
찌개.zip
찜.zip
탕.zip
튀김.zip
한과.zip
해물.zip
회.zip


In [9]:
root_path = DATASET_PATH / "kfood"

zipfile_paths = list(root_path.glob("*.zip"))

for fname in zipfile_paths:
    fname = fname.stem
    dest_path = root_path
    print(fname)
    with zipfile.ZipFile(root_path / "{}.zip".format(fname), "r") as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            member.filename = member.filename.encode('utf8').decode("utf8", "ignore")
            zf.extract(member, dest_path)   


탕
볶음
조림
무침
만두
김치
떡
전골
튀김
구이
음청류
장아찌
장
국
회
찌개
죽
나물
쌈
기타
밥
한과
찜
해물
전
적
면


In [35]:
cat_names = os.listdir(root_path)
cat_names = [root_path / name for name in cat_names if os.path.isdir(os.path.join(root_path, name))]
print(cat_names)

[PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/적'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/기타'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/회'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/밥'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/무침'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/만두'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/구이'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/나물'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/탕'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/쌈'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/국'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/장'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/면'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/전골'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/떡'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/전'), PosixPath('/Users/hwijin/Desktop/Code/ml/KFood/kfood/해물'), PosixP

In [36]:
from PIL import Image
path = root_path / "국/계란국/Img_014_0000.jpg"

with Image.open(path) as image:
    np_image = np.array(image)

print(np_image.shape)


(645, 430, 3)


In [37]:
def create_example_protoc(image, label):
    serialized_image = tf.io.serialize_tensor(image)
    return tf.train.Features(
            feature={
                "image" : tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_image.numpy()])),
                "label" : tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
            }
        )

In [46]:
label_paths = list(root_path.glob("*/*"))
label_paths = [label_path for label_path in label_paths if label_path.is_dir()]

label_name_index_dict = {}
for index, label_path in enumerate(label_paths):
    label_name_index_dict[label_path.stem] = index
print(label_name_index_dict.items())

with open(root_path / 'class_label.csv','w') as f:
    w = csv.writer(f)
    w.writerow(label_name_index_dict.keys())
    w.writerow(label_name_index_dict.values())

dict_items([('떡꼬치', 0), ('편육', 1), ('후라이드치킨', 2), ('콩자반', 3), ('젓갈', 4), ('양념치킨', 5), ('과메기', 6), ('피자', 7), ('육회', 8), ('물회', 9), ('알밥', 10), ('누룽지', 11), ('김치볶음밥', 12), ('김밥', 13), ('비빔밥', 14), ('새우볶음밥', 15), ('유부초밥', 16), ('잡곡밥', 17), ('주먹밥', 18), ('회무침', 19), ('꽈리고추무침', 20), ('도라지무침', 21), ('콩나물무침', 22), ('홍어무침', 23), ('잡채', 24), ('도토리묵', 25), ('만두', 26), ('더덕구이', 27), ('떡갈비', 28), ('갈비구이', 29), ('훈제오리', 30), ('장어구이', 31), ('삼겹살', 32), ('불고기', 33), ('황태구이', 34), ('조기구이', 35), ('조개구이', 36), ('갈치구이', 37), ('고등어구이', 38), ('곱창구이', 39), ('닭갈비', 40), ('미역줄기볶음', 41), ('고사리나물', 42), ('가지볶음', 43), ('애호박볶음', 44), ('숙주나물', 45), ('시금치나물', 46), ('갈비탕', 47), ('매운탕', 48), ('감자탕', 49), ('추어탕', 50), ('곰탕_설렁탕', 51), ('삼계탕', 52), ('보쌈', 53), ('육개장', 54), ('북엇국', 55), ('콩나물국', 56), ('무국', 57), ('계란국', 58), ('미역국', 59), ('떡국_만두국', 60), ('시래기국', 61), ('간장게장', 62), ('양념게장', 63), ('물냉면', 64), ('수제비', 65), ('콩국수', 66), ('막국수', 67), ('열무국수', 68), ('비빔냉면', 69), ('짜장면', 70), ('쫄면', 71), ('칼국수', 72), ('잔치국수', 

In [73]:
from contextlib import ExitStack

tfrecords_path = root_path / "tfrecords"
if not tfrecords_path.exists():
    os.mkdir(tfrecords_path)


def write_tfrecords(filepath, n_shards=1): #파일 조각 개수

    print(filepath.stem, sep=' ')
    paths = [ str(tfrecords_path) + "/{}.tfrecord-{:05d}-{:05d}".format(filepath.stem, index+1, n_shards) for index in range(n_shards)]
    image_crops = get_image_crops(filepath)
    image_crops_index = 0

    image_paths = get_image_paths(filepath)

    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path)) for path in paths]
        for index, image_path in enumerate(image_paths):

            shard = index % n_shards
            with Image.open(image_path) as image:

                tf_image = tf.constant(image)
                if image_path.stem == image_crops[image_crops_index][0]:
                    points = []
                    for point in image_crops[image_crops_index][1].split(","):
                        if point == "":
                            points.append(0)
                        else:
                            points.append(int(point))
                        
                    image_crops_index += 1
                    tf_image = tf.image.crop_to_bounding_box(tf_image, points[1], points[0], points[3], points[2])
                    
                example = create_example_protoc(tf_image, label_name_index_dict[filepath.stem])
                writers[shard].write(example.SerializeToString())
    return paths

def get_image_paths(filepath, image_formats=("*.j*", "*.J*", "*.png", "*.PNG", "*.b*", "*.B*", '*.g*', '*.G*')):
    paths = []
    for image_format in image_formats:
        paths += list(filepath.glob(image_format))
    return sorted(paths)

def get_image_crops(filepath):
    crops = []
    properties = filepath / "crop_area.properties"
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=", 2)
            crops.append([name, crop])
    return crops

In [122]:
print(root_path / "구이/갈비구이/crop_area.properties")
pa = root_path / "구이/갈비구이"
i_paths = get_image_paths(pa)
i_crops = get_image_crops(pa)

a = list(map(int, ",45,".split(",")))
print(a)
with Image.open(i_paths[12]) as image:
    tf_image = tf.constant(image)
    print(tf.shape(tf_image))
    #228,270,431,218
    #45,76,538,404
    #12 0,318,740,556
    tf_image = tf.image.crop_to_bounding_box(tf_image, 318,0,556,740)
    plt.imshow(tf_image.numpy())

/Users/hwijin/Desktop/Code/ml/KFood/kfood/구이/갈비구이/crop_area.properties


ValueError: invalid literal for int() with base 10: ''

In [20]:
for label_path in label_paths:
    print(os.path.getsize(write_tfrecords(label_path)[0]) // 1000000)
    

떡꼬치
911
편육
832
후라이드치킨
832
콩자반
832
젓갈
832
양념치킨
832
과메기
834
피자
832
육회
832
물회
842
알밥
832
누룽지
832
김치볶음밥
832
김밥
832
비빔밥
832
새우볶음밥
832
유부초밥
832
잡곡밥
832
주먹밥
832
회무침
835
꽈리고추무침
832
도라지무침
832
콩나물무침
832
홍어무침
832
잡채
832
도토리묵
834
만두
832
더덕구이
832
떡갈비
832
갈비구이
832
훈제오리
832
장어구이
832
삼겹살
832
불고기
832
황태구이
832
조기구이
832
조개구이
832
갈치구이
832
고등어구이
832
곱창구이
832
닭갈비
832
미역줄기볶음
832
고사리나물
832
가지볶음
832
애호박볶음
832
숙주나물
832
시금치나물
832
갈비탕
832
매운탕
833
감자탕
834
추어탕
834
곰탕_설렁탕
833
삼계탕
834
보쌈
834
육개장
832
북엇국
832
콩나물국
832
무국
832
계란국
832
미역국
832
떡국_만두국
832
시래기국
832
간장게장
832
양념게장
832
물냉면
832
수제비
832
콩국수
832
막국수
832
열무국수
832
비빔냉면
834
짜장면
832
쫄면
832
칼국수
832
잔치국수
832
라면
832
짬뽕
832
곱창전골
832
꿀떡
834
경단
832
송편
832
파전
832
계란말이
833
동그랑땡
832
호박전
832
생선전
832
김치전
832
계란후라이
833
감자전
832
멍게
906
산낙지
886
순대
834
찜닭
834
계란찜
834
닭볶음탕
833
꼬막찜
835
김치찜
832
해물찜
832
갈비찜
832
족발
832
수육
835
깻잎장아찌
833
한과
840
약과
834
약식
836
식혜
833
수정과
832
라볶이
832
고추장진미채볶음
832
멸치볶음
832
제육볶음
832
소세지볶음
834
어묵볶음
832
건새우볶음
832
오징어채볶음
901
두부김치
832
떡볶이
832
주꾸미볶음
833
감자채볶음
832
코다